## ARIMA 모델

<br>

* Autoregressive Integrated Moving Average는 개발된지 오래된 방법으로 시계열 데이터 분석을 위해 이해해야 하는 중요한 모델링 또는 예측 기법

<br>

* Stationary VS Non-stationary time series
    * stationary한 데이터가 stationary특성을 가지고 있는 가정하에 ARIMA 모델을 적용

<br>

* Seasonal VS Non-seasonal ARIMA
    * Non-seasonal ARIMA : ARIMA(p,t,q)
    * Seasonal ARIMA : ARIMA(p,t,q)(P,T,Q)m

<br>

* ARIMA <br>
    * Autoregressive - AR(p)
    * Integrated - I(d)
    * Moving Average - MA(q)

<br> 

### 1. Stationary vs Non-Stationary

> Stationary 데이터의 특성
* 연속되는 숫자들의 평균(mean)이 time invariant (시간에 따라 변하지 않는)
* 연속되는 숫자들의 분산(variance)이 time invariant
* 연속되는 숫자들의 공분산(covariance)이 time invariant
* 평균,분산,공분산이 시점에 따라 달라지지 않음
 
> Stationary Test
* ARIMA 모델은 시계열 데이터가 stationary 특성을 보일 때 효과적으로 데이터가 stationary 특성을 보이는지 확인할 수 있어야 함

> Differencing
* 시계열 데이터가 Non-Stationary 하다면 초기 differencing 작업을 ("Intergrated") 한 번 이상 적용하여 데이터를 stationary 하게 만드는 단계가 필요함


<br>

### 2. Stationary Test

> 데이터가 Stationary한지 검증하는 방법

> Augmented Dickey-Fuller test
* 통계적 시험을 통해 시계열 데이터가 stationary 특성을 보이는지 확인할 수 있음
    

<br>

#### 3. Differencing

> Non-Stationary 데이터는 differencing을 통해 stationary하게 변환

> differencing을 한 데이터에 대해 stationary 할 때 까지 differencing을 반복

> seasonal 데이터일 경우 season을 기준으로 differencing
* 예를 들어 1년 주기의 seasonality를 갖는 월간 데이터에 대해 differencing 할 때, differencing의 시간 단위는 1이 아니라 12로
* seasonal ARIMA 모델의 경우 1차 differencing 후 seasonal differencing 하는것도 흔히 사용하는 방법 